In [9]:
%matplotlib widget
from ipywidgets import VBox, HBox, Text, Button, widgets, Layout
from IPython.display import display
from datetime import datetime, timedelta
import plotly.express as px 
import yfinance as yf
import pandas as pd

In [11]:
class StockGraph():
    def __init__(self):
        self.tickers = {}
        self.lines = []
        self.views = {
            "Raw Data": lambda x: x,
            "Rolling Average": lambda x: x.rolling(rolling.value).mean(),
            "Rolling Variance": lambda x: x.rolling(rolling.value).var(),
            "Rolling Standard Deviation": lambda x: x.rolling(rolling.value).var()**0.5,
            "Rolling Coefficient of Variation": lambda x: (x.rolling(rolling.value).var()**0.5) / (x.rolling(rolling.value).mean())
        }


    def plot_data(self, *args):
        if start_date.value.year > 1900 and end_date.value.year > 1900:
            new_df = pd.DataFrame()
            while self.lines:
                item = self.lines.pop()
                item.remove()
            for tick in self.tickers.keys():
                df = self.tickers[tick]
                start = datetime(start_date.value.year, start_date.value.month, start_date.value.day)
                end = datetime(end_date.value.year, end_date.value.month, end_date.value.day)
                selection = df.loc[(df.index >= start) & (df.index <= end)]
                value = self.views[calculation.value](selection[data_type.value])
                new_df[tick] = value
            fig = px.line(new_df, x='Index', y=df.columns)
            fig.show()
            #self.finish_matplotlib()

    def render_stocks(self, *args):
        if stock_text.value != '':
            ticker = yf.download(stock_text.value.upper(), period="max", progress=False)
            if not ticker.empty:
                self.tickers[stock_text.value.upper()] = ticker
                self.plot_data()
        stock_text.value = ''
        boxes = [self.completed_stock(each) for each in self.tickers]
        tickersWidget.children = ([VBox([VBox(boxes), HBox([stock_text, stock_button])])])


    def completed_stock(self, ticker): 
        tickerField = Text(value=ticker,layout=Layout(width='50%',))
        removeButton = Button(description='Remove', button_style='danger',layout=Layout(width='50%',))
        return HBox([tickerField, removeButton])


    def completed_stock(self, ticker):
        def remove_stock(_):
            if tickerField.value in self.tickers.keys():
                self.tickers.pop(tickerField.value)
                self.render_stocks()
            self.plot_data()
                    
        tickerField = Text(value=ticker,layout=Layout(width='50%',))
        removeButton = Button(description='Remove', button_style='danger',layout=Layout(width='50%',))
        removeButton.on_click(remove_stock)
        stock_view = HBox([tickerField, removeButton])
        return stock_view
    
    def finish_matplotlib(self):
        ax.legend()
        title = f"{calculation.value} of {data_type.value} from {start_date.value.strftime('%Y/%m/%d')}"
        title += f" to {end_date.value.strftime('%Y/%m/%d')}"
        ax.set_title(title)
        ax.relim()
        ax.autoscale()
        fig.canvas.draw_idle()

graph = StockGraph()
"""
fig, ax = plt.subplots(figsize=(10, 5))
fig.canvas.toolbar_position = 'bottom'
"""

# Setting up a basic view with an empty field and a button
stock_text = widgets.Text(value='',layout=Layout(width='50%',))
stock_button = widgets.Button(description='Add',layout=Layout(width='50%',))                                 
stock_button.on_click(graph.render_stocks)
tickersWidget = HBox([stock_text, stock_button])

# Choose data to be shown
calculation = widgets.Dropdown(value="Raw Data", options=graph.views.keys(),layout=Layout(width='auto',))
data_type = widgets.Dropdown(value="Close", options=['Open', 'Close', 'High', 'Low', 'Volume'],layout=Layout(width='auto',))
calculation.observe(graph.plot_data)
data_type.observe(graph.plot_data)
dropdowns = VBox([calculation, data_type])

#Choose dates to be shown
start_date = widgets.DatePicker(value=datetime.today() - timedelta(days=365),disabled=False,layout=Layout(width='auto',))
end_date = widgets.DatePicker(value=datetime.today(),disabled=False,layout=Layout(width='auto',))
start_date.observe(graph.plot_data)
end_date.observe(graph.plot_data)
dates = VBox([start_date, end_date])

#Other Settings for Users
rolling = widgets.Dropdown(options=list(range(2,101)), value=60,description='Rolling:',disabled=False,layout=Layout(width='auto',))
rolling.observe(graph.plot_data)

final_view = VBox([tickersWidget, dropdowns, dates, rolling],layout=Layout(width='20%',))
display(final_view)

